In [1]:
import numpy as np
import pickle 
import cv2

In [3]:
draw = False
rectangles_list = []

try:
    with open('rectangles_list', 'rb') as file:
        rectangles_list = pickle.load(file)
except:
    array = []

def draw_rectangles(event, x, y, flags, param):
    
    if event == cv2.EVENT_LBUTTONDOWN:
        draw = True
        
    elif event == cv2.EVENT_LBUTTONUP:
        draw = False
        rectangles_list.append((x,y,x+35,y+13))
        
        with open('rectangles_list', 'wb') as file:
            pickle.dump(rectangles_list, file)
        
    elif event == cv2.EVENT_RBUTTONUP:
        for r in rectangles_list:
            if r[0] <= x <= r[2] and r[1] <= y <= r[3]:
                rectangles_list.remove(r)
                
        with open('rectangles_list', 'wb') as file:
            pickle.dump(rectangles_list, file)
                
                
img = cv2.imread('carparking.png')
cv2.namedWindow('img')
cv2.setMouseCallback('img', draw_rectangles)

while True:
    
    img_copy = img.copy()
    img_copy = cv2.resize(img_copy, (1200, 650))
    for r in rectangles_list:
        cv2.rectangle(img_copy, (r[2], r[3]), (r[0], r[1]), (0, 255, 0), 1)
        
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
    cv2.imshow('img', img_copy)
        
cv2.destroyAllWindows()

In [30]:
video = cv2.VideoCapture('carparking.mp4')

with open('rectangles_list', 'rb') as file:
    rectangles_list = pickle.load(file)

while True:
    
    ret, frame = video.read()
    
    if ret == False:
        frame = video.set(0, cv2.CAP_PROP_POS_FRAMES)
        
    frame = cv2.resize(frame, (1200, 650))
        
    for i, r in enumerate(rectangles_list):
        cropped_img = frame[r[1]:r[3], r[0]:r[2]]
        gray = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY)
        
        thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 25, 16) 
        
        kernel = np.ones((5,5), np.uint8)
        
        img_erosion = cv2.erode(thresh, kernel, iterations=1) 
        img_dilation = cv2.dilate(img_erosion, kernel, iterations=1)
        occupancy = cv2.countNonZero(img_dilation)
        
        if occupancy < 50:
            color = (0, 255, 0)
            
        elif occupancy > 50:
            color = (0, 0, 255)
        
        cv2.rectangle(frame, (r[2], r[3]), (r[0], r[1]), color, 2)
        
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
    cv2.imshow('img', frame)
        
video.release()
cv2.destroyAllWindows()